In [1]:
#

In [2]:
import json
import time
import numpy
import pandas
import datetime

In [9]:
import torch
from torch import nn

In [14]:
from premodels.prenet import Generale as GGG

In [5]:
from thats_the_stuff import gonet as golags

In [23]:
from mpydge.holy.data_keeper.keeper import Conductor
def gonet(data_set, y_names, removes, categoricals, net, net_init_params, net_fit_params):

    exclude = y_names + removes
    x_names = [x for x in data_set.columns.values if (x not in exclude and 'LAG0' not in x)]
    used_names = x_names + y_names
    data_set = data_set[used_names]

    categorical_columns = categoricals
    numerical_columns = x_names

    outputs = y_names

    if categorical_columns is not None:
        for category in categorical_columns:
            data_set[category] = data_set[category].astype('category')
    if numerical_columns is not None:
        for numeric in numerical_columns:
            data_set[numeric] = data_set[numeric].astype('float64')

    data_set[outputs[0]] = data_set[outputs[0]].astype('category')

    data = Conductor(data_frame=data_set, target=outputs)

    net = net(data=data, **net_init_params)
    print(net)

    net_fit_params['optimiser'] = net_fit_params['optimiser'](net.parameters(), lr=0.001)

    net.fit(**net_fit_params)
    net.fit_plot()

    from sklearn.metrics import mean_squared_error as MSE, r2_score as R2

    summary_params = {'on': 'train',
                      'loss_function': MSE,
                      'score': R2}
    net.summary(**summary_params)

    summary_params = {'on': 'validation',
                      'loss_function': MSE,
                      'score': R2}
    net.summary(**summary_params)

    summary_params = {'on': 'test',
                      'loss_function': MSE,
                      'score': R2}
    net.summary(**summary_params)

    return None

golags = gonet

In [6]:
d = './dataset.csv'
data = pandas.read_csv(d)
data = data.rename(columns={'lag': 'news_horizon'})
data = data.set_index(['ticker', 'time', 'news_horizon'], drop=False)
data = data.sort_index()
data

ticker  \
ticker time                      news_horizon          
MSFT   2019-08-30 15:08:00+00:00 3              MSFT   
                                 3              MSFT   
                                 6              MSFT   
                                 6              MSFT   
                                 9              MSFT   
...                                              ...   
       2019-09-26 20:00:00+00:00 94             MSFT   
                                 94             MSFT   
                                 94             MSFT   
                                 94             MSFT   
                                 97             MSFT   

                                                                    time  \
ticker time                      news_horizon                              
MSFT   2019-08-30 15:08:00+00:00 3             2019-08-30 15:08:00+00:00   
                                 3             2019-08-30 15:08:00+00:00   
                                 6             2019-08-30 15:08:00+00:00   
                                 6             2019-08-30 15:08:00+00:00   
                                 9             2019-08-30 15:08:00+00:00   
...                                                                  ...   
       2019-09-26 20:00:00+00:00 94            2019-09-26 20:00:00+00:00   
                                 94            2019-09-26 20:00:00+00:00   
                                 94            2019-09-26 20:00:00+00:00   
                                 94            2019-09-26 20:00:00+00:00   
                                 97            2019-09-26 20:00:00+00:00   

                                               open_LAG0  close_LAG0  \
ticker time                      news_horizon                          
MSFT   2019-08-30 15:08:00+00:00 3             -0.000292   -0.000438   
                                 3             -0.000292   -0.000438   
                                 6             -0.000292   -0.000438   
                                 6             -0.000292   -0.000438   
                                 9             -0.000292   -0.000438   
...                                                  ...         ...   
       2019-09-26 20:00:00+00:00 94            -0.000072    0.000716   
                                 94            -0.000072    0.000716   
                                 94            -0.000072    0.000716   
                                 94            -0.000072    0.000716   
                                 97            -0.000072    0.000716   

                                               high_LAG0  low_LAG0  \
ticker time                      news_horizon                        
MSFT   2019-08-30 15:08:00+00:00 3             -0.000365 -0.000511   
                                 3             -0.000365 -0.000511   
                                 6             -0.000365 -0.000511   
                                 6             -0.000365 -0.000511   
                                 9             -0.000365 -0.000511   
...                                                  ...       ...   
       2019-09-26 20:00:00+00:00 94            -0.000215  0.000933   
                                 94            -0.000215  0.000933   
                                 94            -0.000215  0.000933   
                                 94            -0.000215  0.000933   
                                 97            -0.000215  0.000933   

                                               volume_LAG0  open_LAG1  \
ticker time                      news_horizon                           
MSFT   2019-08-30 15:08:00+00:00 3                0.539497  -0.000146   
                                 3                0.539497  -0.000146   
                                 6                0.539497  -0.000146   
                                 6                0.539497  -0.000146   
                                 9        

In [24]:
tsi_names = ['news_time']
y_names = ['open_LAG0']
removes = ['ticker', 'time', 'id', 'title', 'news_time']
categoricals = None

net_init_params = {'layers': [nn.Linear], 
                   'layers_dimensions': [data.shape[0]],
                   'activators': [nn.Tanh], 
                   'preprocessor': nn.BatchNorm1d, 
                   'embeddingdrop': 0.0, 
                   'activators_args': {}, 
                   'interprocessors': None,
                   'interdrops': None, 
                   'postlayer': nn.Linear}
net_fit_params = {'loss_function': nn.MSELoss(),
                  'optimiser': torch.optim.Adam,
                  'epochs': 300}


golags(data, y_names, removes, categoricals, GGG, net_init_params, net_fit_params)

<ipython-input-23-3ccc25d3ef2d>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set[numeric] = data_set[numeric].astype('float64')
<ipython-input-23-3ccc25d3ef2d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set[outputs[0]] = data_set[outputs[0]].astype('category')


ValueError: need at least one array to stack